# Boltzmann portfolios

### Part 1: Single decision relationship to Markowitz

We develop an alternative to the traditional mean-variance
framework ("*Markowitz*" portfolios) called
***Boltzmann*** portfolios which addresses uncertainty
from the standpoint of entropy and optimal sequential decisions.
The result is a faster online algorithm which is more robust
and has no dependencies on offline convex optimization packages.

Boltzmann portfolios rely on geometric mean returns since they 
optimally express mean-variance under logarithmic utility. 
Accuracy has been improved by using our research on
Gaussian mixtures presented in https://git.io/gmix --
specifically, our function gemrat() computes the geometric mean
rate by taking into account the fourth central moment, kurtosis.
This is crucial for *leptokurtotic* ("fat-tailed") assets.

After individual geometric mean rates of the underlying assets
have been estimated, we use the covariance matrix to inform
us of possible inter-correlations we can exploit the minimize
the variance of the Boltzmann portfolio.
Such minimization helps to maximize the portfolio's return.

Markowitz portfolios are optimal in the arithmetic mean-variance
framework for a *single-period*. They are fragile to changing
market conditions, much like elegant battle strategies which
crumble under harsh war conditions.
In contrast, Boltzmann portfolios are designed to be
adaptive over multiple periods to maximize final wealth.
Techniques have been borrowed from Bayesian and
reinforcement learning.

[Sequential decision-making using Boltzmann porfolios
will be covered in another notebook (Part ?).]

*Dependencies:*

- Repository: https://github.com/rsvp/fecon235
     
*CHANGE LOG*

    2017-06-28  Refactor subroutines to lib/ys_prtf_boltzmann.py
    2017-06-27  Rewrite relying more arrays than lists, for speed later.
    2017-06-26  Rough draft, test subroutines.

In [1]:
from fecon235.fecon235 import *

In [2]:
#  PREAMBLE-p6.15.1223d :: Settings and system details
from __future__ import absolute_import, print_function, division
system.specs()
pwd = system.getpwd()   # present working directory as variable.
print(" ::  $pwd:", pwd)
#  If a module is modified, automatically reload it:
%load_ext autoreload
%autoreload 2
#       Use 0 to disable this feature.

#  Notebook DISPLAY options:
#      Represent pandas DataFrames as text; not HTML representation:
import pandas as pd
pd.set_option( 'display.notebook_repr_html', False )
from IPython.display import HTML # useful for snippets
#  e.g. HTML('<iframe src=http://en.mobile.wikipedia.org/?useformat=mobile width=700 height=350></iframe>')
from IPython.display import Image 
#  e.g. Image(filename='holt-winters-equations.png', embed=True) # url= also works
from IPython.display import YouTubeVideo
#  e.g. YouTubeVideo('1j_HxD4iLn8', start='43', width=600, height=400)
from IPython.core import page
get_ipython().set_hook('show_in_pager', page.as_hook(page.display_page), 0)
#  Or equivalently in config file: "InteractiveShell.display_page = True", 
#  which will display results in secondary notebook pager frame in a cell.

#  Generate PLOTS inside notebook, "inline" generates static png:
%matplotlib inline   
#          "notebook" argument allows interactive zoom and resize.

 ::  Python 2.7.13
 ::  IPython 5.1.0
 ::  jupyter_core 4.2.1
 ::  notebook 4.1.0
 ::  matplotlib 1.5.1
 ::  numpy 1.11.0
 ::  scipy 0.17.0
 ::  sympy 1.0
 ::  pandas 0.19.2
 ::  pandas_datareader 0.2.1
 ::  Repository: fecon235 v5.17.0603 devPrtf
 ::  Timestamp: 2017-06-27T18:52:12Z
 ::  $pwd: /media/yaya/virt15h/virt/dbx/Dropbox/ipy/fecon235/nb


## Introduction


Traditional portfolio selection (see References) develop
what is generally known as the "**efficient frontier**"
pictured here:  

![Efficient mean-variance frontier](http://webpage.pace.edu/pviswanath/notes/investments/gif/assetalloc77.gif)

Please pay particular attention on the apex of the hyperbolic curve 
(the "bullet") which indicates the point of **global minimum variance**.

A student may first believe the frontier's usefulness
in portfolio management, but later as a professional will
voice something along these lines:

> What I now find problematic is this theoretical concept that one
needs to only pick a point along the top of the bullet to achieve
the optimal balance between risk and reward.
Yeah, like it's so easy to just slide your finger along the curve
and select at your leisure what you'd like to earn on your money.
The focus on past prices that make up these curves
and the assumption that the covariance structure is stationary
in perpetuity is so wrong in practice.
What is rarely talked about is that the future will
NOT fall along that line. It is nice to know what you
could have earned, but what you will earn surely will not be so
easily cherry-picked off of some rear-view historic curve.
--(edited Market Tech comment on 2015-04-10)



## Download data and construct a dataframe

We retrieve the following data of daily frequency
representing equities worldwide and gold by five ETF securities: 

In [9]:
#  Convenient dictionary set in fecon235.py,
#  where key is world region, and value is its fecon235 data code:
world4d

{'America': 's4spy',
 'Emerging': 's4eem',
 'Europe': 's4ezu',
 'Gold': 's4gld',
 'Japan': 's4ewj'}

In [10]:
#  Or manually specify your own dictionary here:
prices_dic = world4d

In [11]:
#  Download data into a dataframe, alphabetically by key:
prices = groupget( prices_dic, maxi=3650 )
#  ... about ten years worth.

 ::  Retrieved from Google Finance: SPY
 ::  Retrieved from Google Finance: EEM
 ::  Retrieved from Google Finance: EZU
 ::  Retrieved from Google Finance: GLD
 ::  Retrieved from Google Finance: EWJ


### ========= Specify DATES for  [start:end] =========

We can analyze different epochs of history
by executing **"Cell" > "Run All Below"** from the Jupyter menu,
*without downloading the data again*.

In [12]:
#  Set the start and end dates:
start = '2007-01-01'
end = '2100-01-01'

### Summary statistics for prices

In [13]:
stats( prices[start:end] )

           America     Emerging       Europe         Gold        Japan
count  2513.000000  2513.000000  2513.000000  2513.000000  2513.000000
mean    156.098743    39.844282    37.192829   120.000330    44.313796
std      43.186322     6.005217     8.096488    26.186519     6.094609
min      68.110000    18.260000    20.320000    64.420000    27.480000
25%     122.760000    37.200000    32.310000   104.080000    39.160000
50%     145.070000    40.620000    35.710000   118.830000    44.880000
75%     198.460000    43.410000    39.440000   133.740000    48.880000
max     244.660000    55.670000    63.440000   184.590000    59.120000

 ::  Index on min:
America    2009-03-09
Emerging   2008-11-20
Europe     2009-03-09
Gold       2007-07-05
Japan      2009-03-09
dtype: datetime64[ns]

 ::  Index on max:
America    2017-06-19
Emerging   2007-10-31
Europe     2007-10-31
Gold       2011-08-22
Japan      2007-07-13
dtype: datetime64[ns]

 ::  Head:
            America  Emerging  Europe   Gold 

## Geometric mean rate

David E. Shaw, famous for his proprietary hedge fund, remarked that 
one of the most important equations in finance is the penalization 
of arithmetic mean by one-half of variance:

$$ g = \mu - \frac{\sigma^2}{2} $$

which turns out to be a second-order approximation of geometric mean rate.
It is good enough to maximize, before considering 
intricate mean-variance trade-offs.

However, many assets have leptokurtotic returns ("fat-tails") and so
a better approximation for the geometric mean rate is needed
which takes into account the fourth central moment called *kurtosis*.
Details are given on Gaussian mixtures in our research
at https://git.io/gmix

For maximimizing wealth over many periods, the geometric mean rate
is a far more reliable metric than the arithmetic mean rate.

A Boltzmann portfolio maximizes the weighted geometric mean rate
of its underlying assets.

The source code shows that groupgemrat() gives us 
the **geometric** mean return, followed by 
the **arithmetic mean return, volatility, and Pearson kurtosis**, 
then yearly frequency, sample size, and key -- in list format.

In [14]:
#  Geometric mean rates, non-overlapping, annualized:
gems = groupgemrat( prices[start:end], yearly=256 )
gems

[[2.08, 4.81, 20.93, 16.09, 256, 2512, 'America'],
 [-11.02, -0.66, 33.25, 17.26, 256, 2512, 'Emerging'],
 [-10.55, -3.95, 29.76, 10.04, 256, 2512, 'Europe'],
 [4.01, 6.1, 19.69, 9.32, 256, 2512, 'Gold'],
 [-5.04, -0.94, 23.66, 16.92, 256, 2512, 'Japan']]

The fourth element in each list gives us the kurtosis statistic
where 3 is theoretically expected if the distribution is Gaussian.

## Portfolio details

Single agent? Just pick the one with highest geometric mean rate.

In [15]:
#  By construction, keys will be alphabetically sorted:
keys = [ item[-1] for item in gems ]
keys

['America', 'Emerging', 'Europe', 'Gold', 'Japan']

In [17]:
#  Gather just the geometric mean rates into an array:
rates = np.array([item[0] for item in gems]).reshape(len(gems), 1)
rates

array([[  2.08],
       [-11.02],
       [-10.55],
       [  4.01],
       [ -5.04]])

In [18]:
#  Compute the covariance matrix:
V = covdiflog( prices[start:end] )

In [19]:
#  The covariance matrix itself is hard to interpret,
#  so show the Pearson correlation coefficients,
#  rounded to n decimal places:
print( cov2cor(V, n=2) )

[[ 1.    0.88  0.87  0.02  0.77]
 [ 0.88  1.    0.84  0.14  0.75]
 [ 0.87  0.84  1.    0.12  0.74]
 [ 0.02  0.14  0.12  1.    0.06]
 [ 0.77  0.75  0.74  0.06  1.  ]]


This correlation matrix pertains to the differential
between logged prices, *not* the prices themselves.

As expected, the equities group is inter-correlated,
whereas Gold rates stand apart.

## Weights from covariance matrix

We now turn our attention to the weights associated with the **Global
Minimum Variance Portfolio**. Its derivation is found in Cochrane
(2005), chp. 5, p.83:

$$ \mathbf{w} = \frac{V^{-1}\mathbf{1}} { \mathbf{1}^\top V^{-1} \mathbf{1} } $$

Note that the weights are solely dependent on the covariance matrix $V$.
There are no constraints involved.

The Boltzmann portfolio is *informed* by the Lagrangian formulation
of the covariance structure, not in the static single-period sense,
but rather in its dynamic evolution over time.
Weights can be revised by an online algorithm which
tracks covariance, e.g. the Kalman filter, without
performing quadratic optimization offline.

We shall take only what we truly need in terms of parameters.

In [21]:
##  Uncomment to see defined function...
#  weighcov??

In [22]:
globalw = weighcov( V )
globalw

array([[ 0.90353325],
       [-0.34943601],
       [-0.19642015],
       [ 0.46541758],
       [ 0.17690532]])

The weights at this point are for the
*Global Minimum Variance Porfolio* (GMVP).
Negative weights imply trading short in the market.

We mark our debt to Harry Markowitz,
see his further work on constrainted portfolios.

## Deciding on the expert(s)

In [23]:
#  Using arrays, we are multiplying element-wise:
scores = globalw * rates
scores

array([[ 1.87934917],
       [ 3.8507848 ],
       [ 2.07223259],
       [ 1.8663245 ],
       [-0.89160283]])

In [24]:
#  GLOBAL portfolio return with UNRESTRICTED short sales:
np.sum(scores)

8.7770882295247112

The weights indicating short positions are usually
paired with negative geometric returns.

Interesting that America had the highest geometric mean return,
but now considering the covariance matrix,
Emerging has the best score (but involving shorting it).

### Dealing with possible short sales

Negative weights imply the underlying assets should be shorted.
A Boltzmann portfolio only considers the weights as ***advisory***.

We may want to limit short sales at -0.3 weight, or perhaps ignore tiny
positions for rebalancing purposes in a multiple-period setting.

So we specify a threshold weight, and renormalize.

In [25]:
MIN_weight = 0.05

In [26]:
trimit?

Signature: trimit(it, floor, level)
Docstring: For an iterable, accept values >= floor, else set to level.
File:      ~/Dropbox/ipy/fecon235/lib/ys_prtf_boltzmann.py
Type:      function


In [27]:
weights = trimit( globalw, MIN_weight, 0 )
weights

array([[ 0.90353325],
       [ 0.        ],
       [ 0.        ],
       [ 0.46541758],
       [ 0.17690532]])

In [28]:
weights = renormalize(weights)
weights

array([[ 0.58448728],
       [ 0.        ],
       [ 0.        ],
       [ 0.30107431],
       [ 0.11443841]])

In [29]:
scores_trim = weights * rates
scores_trim

array([[ 1.21573353],
       [-0.        ],
       [-0.        ],
       [ 1.207308  ],
       [-0.57676959]])

In [30]:
#  Portfolio return with trimmed weights:
np.sum(scores_trim)

1.8462719335475208

## Introducing Boltzmann and softmax

Suppose you have n classes. For some feature, you want to estimate its
probabilities of being in class i. However, your algorithm does not
directly produce probabilities -- instead it first produces real-valued scores.
From these scores you can define the probabilities using the
softmax function. We work with functions that map to the unconstrained space
of scores, and then map those scores to the space of probability vectors.
This allows us to divide up the problem into n subproblems of predicting
scores. One can view this as a generalization of logistic regression.

In statistical physics, the softmax function happens to be the probability of
an atom being found in a quantum state of energy when the atom is part of an
ensemble that has reached thermal equilibrium.
This is known as the **Boltzmann** distribution.

In the field of reinforcement learning, the softmax function is used to
convert values into action probabilities.
A positive parameter T called the temperature is introduced
to divide through each value. It is a scaling operation such that
high temperatures cause corresponding actions to be equi-probable.
Low temperatures cause a greater difference in selection probability
for actions that differ in their value estimates.
At low temperatures, the probability of the action with the
highest expected reward tends to 1.

In [31]:
#  The temperature is a hyperparameter which should be varied:
TEMPERATURE = 55

In [32]:
problist = softmax( scores_trim, temp=TEMPERATURE )[-1]
probs = np.array( problist ).reshape(len(problist), 1)
probs

array([[ 0.4651],
       [ 0.0342],
       [ 0.0342],
       [ 0.4567],
       [ 0.0099]])

Want to calculate new trimmed weights based on softmax adjustments.

EXPLAIN.

In [33]:
trimprobs = weights * probs
trimprobs

array([[ 0.27184503],
       [ 0.        ],
       [ 0.        ],
       [ 0.13750064],
       [ 0.00113294]])

Renormalization is necessary, but also trimit() is helpful,
followed by renormalize() again.

In [34]:
trimprobs = renormalize(trimit(renormalize(trimprobs), MIN_weight, 0))
trimprobs

array([[ 0.66409651],
       [ 0.        ],
       [ 0.        ],
       [ 0.33590349],
       [ 0.        ]])

In [35]:
scores_soft = trimprobs * rates
scores_soft

array([[ 1.38132074],
       [-0.        ],
       [-0.        ],
       [ 1.34697299],
       [-0.        ]])

In [36]:
#  Portfolio return for Boltzmann portfolio:
np.sum(scores_soft)

2.7282937329110535

In [37]:
def boltzmann_cap( capital, probs, rates, keys ):
    '''Recap porfolio numerically -- only within this notebook.'''
    for i, p in enumerate(probs):
        print( keys[i], round(p*capital, 0) )
    print("_________")
    scores = probs * rates
    print("PORTFOLIO geometric mean return:", round(np.sum(scores), 2), "%")

In [39]:
#  Allot across per thousand in capital:
boltzmann_cap(1000, trimprobs, rates, keys)

America 664.0
Emerging 0.0
Europe 0.0
Gold 336.0
Japan 0.0
_________
PORTFOLIO geometric mean return: 2.73 %


## Summary

```
prices ---> cov ---> globalw
  |                    |
  |                  trimit
  |                  renormalize
  |                    |
  |                    |
groupgemrat          weights
  |                    |
  |____________________|
             |
  temp --> softmax --> probs --> trimprobs --> Boltzmann
             
```

---

## References

- John H. Cochrane, 2005 revised ed., *Asset Pricing*, Princeton U. Press.

- On softmax:
    - https://en.wikipedia.org/wiki/Softmax_function 
    - http://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative
    - http://cs231n.github.io/linear-classify/#softmax 
    - https://en.wikipedia.org/wiki/Reinforcement_learning 